In [9]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

In [10]:
## Carboxylic acid

DIAZO = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[C:3]=[N+]=[N-]>>[C:1](=O)[O:2][C:3]')
RED = AllChem.ReactionFromSmarts('[C:1](=O)[O:2]>>[C:1][O:2]')
RLI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O.[C:3][Li]>>[C:1](=[O:2])[C:3]')
CHLORI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O>>[C:1](=[O:2])Cl')
HVZ = AllChem.ReactionFromSmarts('[CH2:1][C:2](=O)O>>[C:1](Br)[C:2](=O)O')

acid_reactions_list = [
    
    ('DIAZO', DIAZO, ['C=[N+]=[N-]'], ['']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]', 'B.C1CCOC1']),
    ('RLI', RLI, ['[Li]C', '[Li]CC', '[Li]CCC', '[Li]CCCC', '[Li]c1ccccc1'], ['']),
    ('CHLORI', CHLORI, [''], ['ClS(Cl)=O']),
    ('HVZ', HVZ, [''], ['BrBr.BrP(Br)Br']),
    
]

In [11]:
import random, copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else cano(smiles)

# choose carboxylic acid (tmp)
acid_smi = 'CC(C)CCC(O)=O'
acid = Chem.MolFromSmiles(acid_smi)

# choose alcohol (tmp)
alcohol_smi = 'CCCBr'

# general reactions
for reaction in acid_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ESTER':
            reagent = alcohol_smi
        if reagent == '': products = reaction[1].RunReactants((acid,))
        else: products = reaction[1].RunReactants((acid, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(acid_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

DIAZO
CC(C)CCC(=O)O.C=[N+]=[N-]>>COC(=O)CCC(C)C
RED
CC(C)CCC(=O)O>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCCO
CC(C)CCC(=O)O>B.C1CCOC1>CC(C)CCCO
RLI
CC(C)CCC(=O)O.[Li]C>>CC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CC>>CCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCC>>CCCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCCC>>CCCCC(=O)CCC(C)C
CHLORI
CC(C)CCC(=O)O>O=S(Cl)Cl>CC(C)CCC(=O)Cl
HVZ
CC(C)CCC(=O)O>BrBr.BrP(Br)Br>CC(C)CC(Br)C(=O)O
